In [1]:
import optim
import PyCall

In [2]:
reload("optim"); reload("tardis")

In [3]:
# define order of linear models
αOrder = 4
βOrder = 2

# read in data and create posterior
dim = αOrder + βOrder
∇res, Hres = optim.allocations(dim)
frame, (P, ∇P!, HP!) = optim.problem(αOrder=αOrder, βOrder=βOrder, run=99, npackets=10000);

In [4]:
# compute the posterior mode to estimate the evidence
@time maxP, posterior_mode, ret = optim.run_nlopt(frame, P, ∇P!, HP!, αOrder, βOrder, xtol_rel=1e-3)
HP!(posterior_mode, Hres)
evidence = optim.laplace(maxP, Hres)
println("got $maxP at $(posterior_mode) (returned $ret) and evidence $evidence")

α constraint at the mode 1.0
β constraint at the mode -7.14383370729536e-310
  1.511398 seconds (18.95 M allocations: 301.509 MB, 1.83% gc time)
got 20602.560751125056 at [1.6,-0.0166903,0.00291387,-0.00173988,63.0231,2.00966] (returned XTOL_REACHED) and evidence 20594.954465590723


 17.973554 seconds (448.23 M allocations: 6.679 GB, 7.65% gc time)
got 207090.19544171615 at [1.59799,-0.279475,0.0137261,0.0925643,56.8278,2.24986] (returned XTOL_REACHED) and evidence 207075.6746506941

In [5]:
# the posterior mean with the normalized posterior
pm = optim.PosteriorMean(0.1, 0.2, 6.0)
Pmean, ∇Pmean!, HPmean! = optim.targetfactory(frame, αOrder, βOrder, evidence=evidence, pm=pm)
@time maxPmean, mean_mode, ret = optim.run_nlopt(frame, Pmean, ∇Pmean!, HPmean!, αOrder, βOrder,
init=posterior_mode, xtol_rel=1e-3)
println("max of normalized posterior = $maxPmean at $(mean_mode), returned $ret")

α constraint at the mode 1.0
β constraint at the mode 0.0
  0.928148 seconds (16.54 M allocations: 253.491 MB, 1.69% gc time)
max of normalized posterior = Inf at [1.6,-0.0166903,0.00291387,5.33411,63.0231,2.00966], returned XTOL_REACHED


 11.617350 seconds (322.92 M allocations: 4.812 GB, 8.23% gc time)
max of normalized posterior = 593.6109048238263 at [1.59822,0.0768632,69.7916,-0.869807], returned XTOL_REACHED (1e-4)


requires the mode from posterior (evidence calculation)

In [6]:

nb = optim.NegBinom(10, 0.5)
Pall, ∇Pall!, HPall! = optim.targetfactory(frame, αOrder, βOrder, evidence=evidence, pm=pm, nb=nb)
@time maxPall, mode, ret = optim.run_nlopt(frame, Pall, ∇Pall!, HPall!, αOrder, βOrder,
init=vcat(posterior_mode, 10), xtol_rel=1e-3, nb=nb)
println("max of normalized all posterior = $maxPall at $mode")


α constraint at the mode 1.0
β constraint at the mode -7.1438343822273e-310
  1.819033 seconds (30.62 M allocations: 470.899 MB, 1.36% gc time)
max of normalized all posterior = 85.51894379311966 at [1.59758,-0.284452,0.126032,0.03427,68.8498,1.91844,10.0018]


In [7]:
points = collect(linspace(0.0, 0.6, 25))
results = zeros(points)
# skip X = 0
optimize = false
for i in 2:length(points)
    pm.X = points[i]
    results[i] = optim.predict_small(frame, Pmean, ∇Pmean!, HPmean!, pm, αOrder, βOrder, posterior_mode,
    nb, optimize=optimize)
end

# self-normalize estimates through Simpson's rule
PyCall.@pyimport scipy.integrate as si
norm = si.simps(results, points)
println("norm = $norm")
results /= norm
println(results)


initial N=3
initial N=4
initial N=5
initial N=6
initial N=6
initial N=7
initial N=8
initial N=9
initial N=10
initial N=10
initial N=11
initial N=12
initial N=13
initial N=13
initial N=14
initial N=15
initial N=16
initial N=16
initial N=17
initial N=18
initial N=18
initial N=19
initial N=20
initial N=20
norm = 0.9706631633076218
[0.0,0.35049,0.770405,1.2962,1.85458,2.37534,2.79684,3.08383,3.22676,3.22964,3.11655,2.91088,2.64597,2.34511,2.03518,1.72718,1.44444,1.1878,0.962158,0.770336,0.608165,0.475484,0.366416,0.280763,0.212839]


Output without normalizing gamma distribution to $x \in [0,1]$

    norm = 0.9632725771703211
    [0.0,0.331754,0.712386,1.18703,1.6955,2.17674,2.58467,2.88045,3.05146,3.10053,3.03929,2.88574,2.67116,2.41638,2.1392,1.85987,1.58856,1.3372,1.10664,0.907021,0.733854,0.587862,0.465607,0.364297,0.283832]

In [8]:
import Plots

In [9]:
Plots.plot(points, results)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 0.4 
 
 
 0.5 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 12.8259,384.952 37.1274,343.603 61.429,294.064 85.7305,232.033 110.032,166.159 134.334,104.723 158.635,54.997 182.937,21.1395 207.238,4.27692 231.54,3.93701 
 255.841,17.2791 280.143,41.5427 304.444,72.7951 328.746,108.289 353.048,144.853 377.349,181.189 401.651,214.545 425.952,244.822 450.254,271.442 474.555,294.072 
 498.857,313.204 523.158,328.857 547.46,341.724 571.761,351.829 596.063,359.842 
 "/>
 
 
 
 
 y1

In [10]:
frame[:energies] |> maximum

0.18642974474027474

Min and max $\nu$

In [11]:
frame[:nus][1], frame[:nus][end]

(0.03288078826335864,3.2303291975036594)

In [12]:
using Polynomials,LaTeXStrings

In [13]:
αPoly = Poly(posterior_mode[1:αOrder])
Plots.plot(frame[:nus], polyval(αPoly, frame[:nus]), xlabel=L"\nu", ylabel=L"\alpha")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 1.525 
 
 
 1.550 
 
 
 1.575 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 63.6592,3.93701 64.1222,4.14257 66.8577,5.3533 67.0097,5.42034 68.852,6.23192 68.8906,6.24886 69.0667,6.32628 69.3988,6.47221 69.611,6.56541 70.624,7.00982 
 70.6914,7.03932 70.865,7.11541 71.2013,7.26268 71.368,7.33564 71.783,7.51719 71.8408,7.54247 72.1399,7.67326 72.3842,7.77998 72.4911,7.82666 72.6062,7.87692 
 72.7105,7.92248 72.7226,7.92775 72.8051,7.96379 72.9274,8.01717 73.1121,8.09775 74.1855,8.56559 74.5553,8.72654 74.5785,8.73665 74.8367,8.84898 74.9355,8.89194 
 75.754,9.24759 75.7621,9.2511 76.2902,9.48034 76.4569,9.55261 76.8546,9.72505 77.194,9.87209 77.4597,9.98714 77.6318,10.0616 77.6754,10.0805 77.8334,10.1489 
 77.8766,10.1676 77.8868,10.172 78.4382,10.4104 78.5324,10.4511 78.5752,10.4696 78.661,10.5067 78.7471,10.5439 78.8391,10.5836 78.9228,10.6198 79.0633,10.6805 
 79.3736,10.8144 79.4736,10.8575 79.8664,11.0269 80.1383,11.1441 80.3106,11.2184 80.4424,11.2751 80.5718,11.3309 80.6222,11.3526 80.8406,11.4466 80.8593,11.4547 
 81.2208,11.6102 81.2711,11.6319 81.352,11.6667 81.6855,11.8101 81.8204,11.868 81.9573,11.9269 82.1649,12.0161 82.1679,12.0173 82.2173,12.0386 82.2762,12.0639 
 82.41,12.1213 82.4417,12.1349 82.4987,12.1594 82.5532,12.1828 82.5943,12.2004 82.8345,12.3035 82.9498,12.353 83.0417,12.3924 83.1047,12.4194 83.1411,12.435 
 83.4244,12.5565 83.5782,12.6224 83.6291,12.6442 83.6492,12.6528 83.999,12.8027 84.0463,12.8229 84.1529,12.8686 84.1754,12.8782 84.2309,12.902 84.3631,12.9586 
 84.4728,13.0055 84.8269,13.157 84.8328,13.1596 84.9205,13.1971 84.9643,13.2158 85.0104,13.2355 85.0671,13.2597 85.1721,13.3046 85.2314,13.33 85.3275,13.3711 
 85.3672,13.388 85.3681,13.3884 85.4923,13.4415 85.7236,13.5403 85.7864,13.5671 85.7927,13.5698 85.8356,13.5881 85.8373,13.5888 85.9985,13.6577 86.0862,13.6951 
 86.2205,13.7524 86.234,13.7582 86.2522,13.7659 86.3181,13.794 86.3628,13.8131 86.3688,13.8157 86.6793,13.9481 86.7425,13.9751 86.9128,14.0477 87.0403,14.102 
 87.0468,14.1048 87.0912,14.1237 87.138,14.1437 87.2048,14.1722 87.2608,14.196 87.3035,14.2142 87.3579,14.2374 87.3626,14.2394 87.4302,14.2682 87.5408,14.3153 
 87.5424,14.316 87.6549,14.3639 87.6596,14.3659 87.6925,14.3799 87.724,14.3933 87.806,14.4282 87.8878,14.4631 87.9517,14.4903 88.0781,14.5441 88.1578,14.578 
 88.2037,14.5975 88.3295,14.651 88.4178,14.6886 88.5663,14.7518 88.5701,14.7534 88.784,14.8443 88.9704,14.9236 88.9787,14.9271 88.9835,14.9291 89.0152,14.9426 
 89.0276,14.9478 89.2137,15.0269 89.2328,15.0351 89.2378,15.0372 89.2683,15.0501 89.3138,15.0695 89.3484,15.0841 89.4385,15.1224 89.5971,15.1898 89.6164,15.198 
 89.6399,15.208 89.6416,15.2087 89.7311,15.2467 89.769,15.2627 89.7978,15.275 89.8713,15.3062 89.9022,15.3193 89.9851,15.3545 90.1859,15.4397 90.2139,15.4516 
 90.2834,15.481 90.3277,15.4998 90.3428,15.5062 90.3864,15.5247 90.4437,15.549 90.5633,15.5997 90.6807,15.6495 90.6869,15.6521 90.7641,15.6849 90.7842,15.6934 
 90.7882,15.6951 90.8434,15.7185 90.9234,15.7524 90.9767,15.775 91.0205,15.7936 91.1202,15.8358 91.1939,15.867 91.1946,15.8673 91.2022,15.8705 91.2988,15.9115 
 91.4055,15.9566 91.4078,15.9576 91.4259,15.9653 91.6397,16.0558 91.6607,16.0647 91.7396,16.0981 91.8356,16.1387 91.9035,16.1674 91.9286,16.1781 91.9457,16.1853 
 92.0699,16.2379 92.1818,16.2852 92.2016,16.2936 92.2082,16.2964 92.2471,16.3128 92.3294,16.3476 92.4225,16.387 92.4659,16.4054 92.4951,16.4177 92.5121,16.4249 
 92.6388,16.4785 92.6651,16.4896 92.8293,16.559 92.8584,16.5713 93.0024,16.6321 93.0099,16.6353 93.014,16.6371 93.1059,16.6759 93.1097,16.6775 93.1916,16.7121 
 93.2016,16.7163 93.2243,16.7259 93.2306,16.7286 93.3085,16.7615 93.4754,16.832 93.492,16.839 93.5733,16.8733 93.6637,16.9115 93.7083,16.9303 93.7166,16.9338 
 93.735,16.9415 93.736,16.942 93.7

In [14]:
βPoly = Poly(posterior_mode[αOrder+1:αOrder+βOrder])
Plots.plot(frame[:nus], polyval(βPoly, frame[:nus]),  xlabel=L"\nu", ylabel=L"\beta")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 64 
 
 
 66 
 
 
 68 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 36.9926,369.674 37.4787,369.356 40.3513,367.477 40.5108,367.373 42.4455,366.107 42.4859,366.08 42.6709,365.959 43.0196,365.731 43.2424,365.586 44.3062,364.89 
 44.3769,364.843 44.5593,364.724 44.9124,364.493 45.0874,364.379 45.5232,364.093 45.5839,364.054 45.898,363.848 46.1545,363.68 46.2668,363.607 46.3876,363.528 
 46.4972,363.456 46.5098,363.448 46.5965,363.391 46.725,363.307 46.9189,363.18 48.0461,362.443 48.4343,362.189 48.4587,362.173 48.7299,361.996 48.8336,361.928 
 49.6931,361.366 49.7016,361.36 50.2562,360.997 50.4312,360.883 50.8489,360.609 51.2053,360.376 51.4842,360.194 51.665,360.076 51.7108,360.046 51.8767,359.937 
 51.922,359.907 51.9328,359.9 52.5117,359.522 52.6107,359.457 52.6556,359.428 52.7457,359.369 52.8362,359.309 52.9327,359.246 53.0207,359.189 53.1682,359.092 
 53.494,358.879 53.599,358.81 54.0115,358.54 54.297,358.354 54.478,358.235 54.6163,358.145 54.7522,358.056 54.8052,358.021 55.0345,357.871 55.0541,357.858 
 55.4337,357.61 55.4866,357.576 55.5715,357.52 55.9217,357.291 56.0633,357.198 56.2071,357.104 56.4251,356.962 56.4283,356.96 56.4801,356.926 56.542,356.885 
 56.6825,356.793 56.7158,356.771 56.7757,356.732 56.8328,356.695 56.876,356.667 57.1283,356.502 57.2494,356.422 57.3459,356.359 57.412,356.316 57.4502,356.291 
 57.7478,356.096 57.9092,355.991 57.9626,355.956 57.9838,355.942 58.351,355.702 58.4007,355.669 58.5127,355.596 58.5364,355.58 58.5946,355.542 58.7334,355.452 
 58.8487,355.376 59.2204,355.133 59.2267,355.129 59.3188,355.069 59.3648,355.038 59.4132,355.007 59.4727,354.968 59.583,354.896 59.6453,354.855 59.7462,354.789 
 59.7878,354.762 59.7888,354.761 59.9192,354.676 60.1621,354.517 60.228,354.474 60.2346,354.469 60.2797,354.44 60.2815,354.439 60.4508,354.328 60.5428,354.268 
 60.6838,354.176 60.6981,354.166 60.7172,354.154 60.7863,354.109 60.8333,354.078 60.8396,354.074 61.1656,353.86 61.232,353.817 61.4109,353.7 61.5447,353.612 
 61.5515,353.608 61.5981,353.577 61.6473,353.545 61.7174,353.499 61.7762,353.461 61.8211,353.432 61.8783,353.394 61.8832,353.391 61.9542,353.345 62.0703,353.269 
 62.072,353.267 62.1901,353.19 62.1951,353.187 62.2296,353.164 62.2627,353.143 62.3488,353.086 62.4347,353.03 62.5017,352.986 62.6345,352.899 62.7182,352.845 
 62.7664,352.813 62.8985,352.727 62.9912,352.666 63.1472,352.564 63.1511,352.561 63.3757,352.415 63.5715,352.286 63.5802,352.281 63.5852,352.278 63.6186,352.256 
 63.6316,352.247 63.827,352.119 63.8471,352.106 63.8523,352.103 63.8843,352.082 63.9321,352.051 63.9684,352.027 64.0631,351.965 64.2296,351.856 64.2499,351.843 
 64.2746,351.827 64.2763,351.825 64.3703,351.764 64.4101,351.738 64.4404,351.718 64.5175,351.668 64.55,351.646 64.6371,351.589 64.8479,351.452 64.8773,351.432 
 64.9502,351.385 64.9968,351.354 65.0127,351.344 65.0584,351.314 65.1186,351.274 65.2442,351.192 65.3674,351.112 65.374,351.107 65.4551,351.054 65.4761,351.04 
 65.4803,351.038 65.5383,351 65.6223,350.945 65.6783,350.908 65.7243,350.878 65.8289,350.81 65.9064,350.759 65.9071,350.759 65.915,350.753 66.0166,350.687 
 66.1286,350.614 66.131,350.612 66.15,350.6 66.3745,350.453 66.3965,350.438 66.4794,350.384 66.5802,350.318 66.6515,350.272 66.6778,350.254 66.6959,350.243 
 66.8263,350.157 66.9437,350.08 66.9646,350.067 66.9715,350.062 67.0124,350.036 67.0987,349.979 67.1965,349.915 67.2421,349.885 67.2728,349.865 67.2906,349.853 
 67.4236,349.766 67.4512,349.748 67.6237,349.636 67.6542,349.616 67.8054,349.517 67.8133,349.512 67.8176,349.509 67.9142,349.446 67.9181,349.443 68.0042,349.387 
 68.0146,349.38 68.0384,349.364 68.0451,349.36 68.1269,349.306 68.3022,349.192 68.3196,349.18 68.4049,349.125 68.4999,349.062 68.5467,349.032 68.5555,349.026 
 68.5747,349.013 68.5758,349.013 68.579,349.011 6

In [15]:
Plots.histogram(frame[:nus])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 0 
 
 
 200 
 
 
 400 
 
 
 600 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

In [16]:
import GSL, StatsFuns

In [17]:
α=1.5; β=50
x=lgamma(α); y=log(GSL.sf_gamma_inc(α, β))
println(x," , ",y)
println(exp(x))
println(exp(x)-exp(y))
StatsFuns.logsumexp(lgamma(α), -log(GSL.sf_gamma_inc(α, β)))

-0.12078223763524522 , -48.034134349625376
0.886226925452758
0.886226925452758


48.034134349625376

In [18]:
StatsFuns.logsumexp(1,-1)

1.1269280110429725

In [19]:
exp(1)

2.718281828459045

In [20]:
log(exp(1)-exp(1))

-Inf